# Prompt Tuning 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## 学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("/root/PEFT代码/tuning/lesson01/data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:1]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'],
 'input': [''],
 'instruction': ['保持健康的三个提示。']}

## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

## Step4 创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/4e/fc/4efc30dd0e6149cbaa64ae302d553cf7ca0bdf19f3eda8d0004b46e8b7046621/93cfe5a940ccff0ea907978e05a0f2059c81f07b54f83886a76571b4fe1d87ea?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1715406767&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTQwNjc2N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80ZS9mYy80ZWZjMzBkZDBlNjE0OWNiYWE2NGFlMzAyZDU1M2NmN2NhMGJkZjE5ZjNlZGE4ZDAwMDRiNDZlOGI3MDQ2NjIxLzkzY2ZlNWE5NDBjY2ZmMGVhOTA3OTc4ZTA1YTBmMjA1OWM4MWYwN2I1NGY4Mzg4NmE3NjU3MWI0ZmUxZDg3ZWE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=RV9AIh0C2jT6FZJQkH1PuTMgsRxytqiTHYkOh90DXVb3vWgx7E-GTlTsGIG5R4uKjRhCX7qGnR9nuoE-8QnImdqGWdOQgIL0PmEFWDffmaJrATrix63%7ECvGzj7938xfrzf-32ccH3N4lvCi1MLDCII%7E9pUDF-XvEFediRjcf9NSvnBu0-ef3he26JG3WPuKusH00MMdY4zo02cmb%7EOlo50AaTCOau22DEKBXn2Df-jFn

model.safetensors:   8%|8         | 451M/5.59G [00:00<?, ?B/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(901439271 bytes read, 4238174103 more expected)', IncompleteRead(901439271 bytes read, 4238174103 more expected))

## Prompt tuning

### PEFT Step1 配置文件

In [9]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

# Soft Prompt演示

config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
config
# Hard Prompt演示
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
#                             prompt_tuning_init=PromptTuningInit.TEXT,
#                             prompt_tuning_init_text="下面是一段人与智能助手的对话。",
#                             num_virtual_tokens=len(tokenizer("下面是一段人与智能助手的对话。")["input_ids"]),
#                             tokenizer_name_or_path="Langboat/bloom-1b4-zh")
# config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.RANDOM: 'RANDOM'>, prompt_tuning_init_text=None, tokenizer_name_or_path=None)

### PEFT Step2 创建模型

In [10]:
model = get_peft_model(model, config)

In [11]:
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [13]:
model.print_trainable_parameters()

trainable params: 20,480 || all params: 1,303,132,160 || trainable%: 0.0015715980795071467


## Step5 配置训练参数

In [13]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/tuningdata/prompt_tuning2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [14]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [15]:
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.864700
20,2.797400
30,2.857900
40,2.815000
50,2.826200
60,2.836800
70,2.817200
80,2.894000
90,2.817000
100,2.773500


KeyboardInterrupt: 

## Step8 模型推理

In [21]:
from peft import PeftModel
from transformers import pipeline

In [23]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

p_model = PeftModel.from_pretrained(model=model, model_id="/root/autodl-tmp/tuningdata/prompt_tuning/checkpoint-500")

pipe = pipeline("text-generation", model=p_model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Refor

[{'generated_text': 'Human: 如何写好一个简历？\n\nAssistant: 问一个与面试官聊得来的问题。面试官会问到与被面试者经历相似的经历，比如被培训等。你可以就这个话题谈谈。比如你在哪里学习呢？你的父母是谁？你的童年有什么特殊的经历？你的爱好是什么？\nA: 一般来说，面试之前都会有试探性发言。比如你问一些可以谈得来的话题，比如关于你的公司和其他人，或者与公司的合作。你也可以与面试官讨论一些工作以外的事情，比如你从事这个职业的时间点，你认为它对你来说如何？你觉得自己最大的优势是什么？你想学什么，你想要学什么？你为什么想要学这些？\nA: 一份好的简历通常会被评阅的比较详细。所以当面试官问你的工作经历的细节时，要记得多回答一些细节。比如说你的培训？你的工作经历和学习经历如何？还有你参与的学习项目？你觉得工作与学习的平衡点是什么？\nA: 首先不要忘记去面试官的公司，和面试官聊聊。比如聊聊你在当地、本专业和其他方面的地位、你所在公司的声誉以及你期望与公司合作的方式。你也可以问一两个面试官的面试经验，比如你做过面试官会问'}]